In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# Print dataset info
print(f"Number of training samples: {mnist_info.splits['train'].num_examples}")
print(f"Number of test samples: {mnist_info.splits['test'].num_examples}")

# Calculate the number of validation and test samples
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Scale the data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Verify scaling
for image, label in scaled_train_and_validation_data.take(1):
    print(f"Scaled image shape: {image.numpy().shape}, label: {label.numpy()}")

# Shuffle and split the data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Verify dataset sizes
print(f"Validation dataset size: {len(list(validation_data))}")
print(f"Training dataset size: {len(list(train_data))}")

# Batch the data
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(BATCH_SIZE)

# Verify batching
for images, labels in train_data.take(1):
    print(f"Batch of training images shape: {images.numpy().shape}")
    print(f"Batch of training labels shape: {labels.numpy().shape}")

# Get validation inputs and targets
validation_inputs, validation_targets = next(iter(validation_data))
print(f"Validation inputs shape: {validation_inputs.shape}")
print(f"Validation targets shape: {validation_targets.shape}")

# Define the model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
NUM_EPOCHS = 5
history = model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Number of training samples: 60000
Number of test samples: 10000
Scaled image shape: (28, 28, 1), label: 4
Validation dataset size: 6000
Training dataset size: 54000
Batch of training images shape: (100, 28, 28, 1)
Batch of training labels shape: (100,)
Validation inputs shape: (6000, 28, 28, 1)
Validation targets shape: (6000,)


D:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │          39,250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             510 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 42,310 (165.27 KB)

 Trainable params: 42,310 (165.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
540/540 - 7s - 12ms/step - accuracy: 0.8790 - loss: 0.4245 - val_accuracy: 0.9375 - val_loss: 0.2139
Epoch 2/5
540/540 - 4s - 7ms/step - accuracy: 0.9474 - loss: 0.1815 - val_accuracy: 0.9530 - val_loss: 0.1601
Epoch 3/5
540/540 - 4s - 8ms/step - accuracy: 0.9585 - loss: 0.1388 - val_accuracy: 0.9637 - val_loss: 0.1260
Epoch 4/5
540/540 - 4s - 7ms/step - accuracy: 0.9669 - loss: 0.1132 - val_accuracy: 0.9697 - val_loss: 0.1125
Epoch 5/5
540/540 - 4s - 7ms/step - accuracy: 0.9716 - loss: 0.0952 - val_accuracy: 0.9718 - val_loss: 0.0992
100/100 - 1s - 11ms/step - accuracy: 0.9659 - loss: 0.1094
Test Loss: 0.10940918326377869
Test Accuracy: 0.9659000039100647
